In [1]:
## This file implements neural networks and random forest on p0017SpresabsSTCC_qual.
## Due to the imbalanced data and limited cases for class 2, we implement the over-sampling method.
## For fully-connected neural networks, the accuracy is 92.39% for over-sampling data and 90.86% with dropout and regularization.
## For random forest, the accuracy is 96.95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 93.01% for over-sampling data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0017SpresabsSTCC_qual.csv')
df.shape

(255, 148)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      1
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     1
17     1
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     1
      ..
225    0
226    0
227    1
228    0
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    1
245    0
246    0
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    218
1     35
2      2
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,ST,CC,pheno
0,107,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 147)

In [10]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,...,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,ST,CC,pheno
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,1
3,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
4,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 146) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218), (2, 218)]


Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
############# Fully-Connected Neural Network ################

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [16]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [17]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 325us/step - loss: 3.2248 - accuracy: 0.3195 - val_loss: 1.4198 - val_accuracy: 0.3401
Epoch 2/100
457/457 [==============================] - 0s 67us/step - loss: 1.2843 - accuracy: 0.4464 - val_loss: 0.9964 - val_accuracy: 0.5279
Epoch 3/100
457/457 [==============================] - 0s 63us/step - loss: 0.9125 - accuracy: 0.7199 - val_loss: 0.6814 - val_accuracy: 0.7462
Epoch 4/100
457/457 [==============================] - 0s 75us/step - loss: 0.6833 - accuracy: 0.6565 - val_loss: 0.5691 - val_accuracy: 0.7411
Epoch 5/100
457/457 [==============================] - 0s 77us/step - loss: 0.5555 - accuracy: 0.7746 - val_loss: 0.5950 - val_accuracy: 0.7513
Epoch 6/100
457/457 [==============================] - 0s 79us/step - loss: 0.5472 - accuracy: 0.8162 - val_loss: 0.7158 - val_accuracy: 0.8020
Epoch 7/100
457/457 [==============================] - 0s 79us/step - loss: 0.5253 - accu

Epoch 57/100
457/457 [==============================] - 0s 56us/step - loss: 0.1452 - accuracy: 0.9519 - val_loss: 0.2245 - val_accuracy: 0.9239
Epoch 58/100
457/457 [==============================] - 0s 51us/step - loss: 0.1519 - accuracy: 0.9584 - val_loss: 0.2287 - val_accuracy: 0.9289
Epoch 59/100
457/457 [==============================] - 0s 53us/step - loss: 0.1576 - accuracy: 0.9431 - val_loss: 0.2291 - val_accuracy: 0.9239
Epoch 60/100
457/457 [==============================] - 0s 55us/step - loss: 0.1424 - accuracy: 0.9497 - val_loss: 0.2227 - val_accuracy: 0.9442
Epoch 61/100
457/457 [==============================] - 0s 50us/step - loss: 0.1491 - accuracy: 0.9562 - val_loss: 0.2206 - val_accuracy: 0.9239
Epoch 62/100
457/457 [==============================] - ETA: 0s - loss: 0.0833 - accuracy: 1.00 - 0s 49us/step - loss: 0.1607 - accuracy: 0.9453 - val_loss: 0.2216 - val_accuracy: 0.9391
Epoch 63/100
457/457 [==============================] - 0s 56us/step - loss: 0.1644 - ac

In [19]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

197/197 [==============================] - 0s 72us/step
over-sampling test accuracy: 92.39%


In [20]:
#### add dropout and regularizer
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [21]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 419us/step - loss: 12.4846 - accuracy: 0.4026 - val_loss: 9.9383 - val_accuracy: 0.5482
Epoch 2/100
457/457 [==============================] - 0s 71us/step - loss: 10.6719 - accuracy: 0.5164 - val_loss: 7.7851 - val_accuracy: 0.5127
Epoch 3/100
457/457 [==============================] - 0s 71us/step - loss: 9.4695 - accuracy: 0.5624 - val_loss: 6.4051 - val_accuracy: 0.6497
Epoch 4/100
457/457 [==============================] - 0s 70us/step - loss: 8.5720 - accuracy: 0.5098 - val_loss: 5.3466 - val_accuracy: 0.7157
Epoch 5/100
457/457 [==============================] - 0s 69us/step - loss: 7.8174 - accuracy: 0.6039 - val_loss: 4.6412 - val_accuracy: 0.5279
Epoch 6/100
457/457 [==============================] - 0s 69us/step - loss: 7.0489 - accuracy: 0.6193 - val_loss: 4.1300 - val_accuracy: 0.7259
Epoch 7/100
457/457 [==============================] - 0s 75us/step - loss: 6.6414 - ac

457/457 [==============================] - 0s 64us/step - loss: 1.9290 - accuracy: 0.7287 - val_loss: 0.8682 - val_accuracy: 0.8832
Epoch 58/100
457/457 [==============================] - 0s 63us/step - loss: 1.6305 - accuracy: 0.7462 - val_loss: 0.8880 - val_accuracy: 0.8680
Epoch 59/100
457/457 [==============================] - 0s 60us/step - loss: 1.6629 - accuracy: 0.7374 - val_loss: 0.7503 - val_accuracy: 0.8883
Epoch 60/100
457/457 [==============================] - 0s 60us/step - loss: 1.4447 - accuracy: 0.7527 - val_loss: 0.7378 - val_accuracy: 0.8934
Epoch 61/100
457/457 [==============================] - 0s 59us/step - loss: 1.5071 - accuracy: 0.7571 - val_loss: 0.6849 - val_accuracy: 0.9086
Epoch 62/100
457/457 [==============================] - 0s 64us/step - loss: 1.5674 - accuracy: 0.7352 - val_loss: 0.7088 - val_accuracy: 0.9137
Epoch 63/100
457/457 [==============================] - 0s 60us/step - loss: 1.5738 - accuracy: 0.7571 - val_loss: 0.7654 - val_accuracy: 0.883

In [23]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

197/197 [==============================] - 0s 72us/step
over-sampling test accuracy: 90.86%


In [24]:
############## Random Forest ##############

In [25]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 96.95%


In [27]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.92473118 0.92473118 0.92307692 0.94444444 0.93333333]
0.9300634132892197
